In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
#Importing dataset
movies = pd.read_csv('ml-1m/movies.dat',sep='::',header=None, engine='python', encoding='latin-1')
users = pd.read_csv('ml-1m/users.dat',sep='::',header=None, engine='python', encoding='latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat',sep='::',header=None, engine='python', encoding='latin-1')

In [3]:
#Creating training and test set
training_set = pd.read_csv('ml-100k/u1.base',delimiter='\t')
training_set = np.array(training_set,dtype='int')

test_set = pd.read_csv('ml-100k/u1.test',delimiter='\t')
test_set = np.array(test_set,dtype='int')

In [4]:
#Number of users and movies
nb_users = max(max(training_set[:,0]), max(test_set[:,0]))
nb_movies = max(max(training_set[:,1]), max(test_set[:,1]))

In [5]:
#Converting data to array with users in lines and moviews in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users+1):
        id_movies = data[:,1][data[:,0]==id_users]
        id_ratings = data[:,2][data[:,0]==id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] =id_ratings
        new_data.append(list(ratings))
    return new_data

training_set = convert(training_set)
test_set = convert(test_set)

In [6]:
#Converting to torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)


In [7]:
#Creating a AE class
class SAE(nn.Module):
    
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20) #Getting 20 hidden features
        self.fc2 = nn.Linear(20,10) #This is similar to neurons added in DL
        self.fc3 = nn.Linear(10,20)
        self.fc4 = nn.Linear(20,nb_movies)
        self.activation = nn.Sigmoid()
    
    def forward(self,x):
        x= self.activation(self.fc1(x))
        x= self.activation(self.fc2(x))
        x= self.activation(self.fc3(x))
        x = self.fc4(x)
        return x        

In [8]:
sae = SAE()

In [9]:
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay = 0.5)

In [ ]:
#Training SAE
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch: '+str(epoch)+'loss: '+ str(train_loss/s))
            

epoch: 1loss: tensor(1.7717)
epoch: 2loss: tensor(1.0966)
epoch: 3loss: tensor(1.0533)
epoch: 4loss: tensor(1.0385)
epoch: 5loss: tensor(1.0309)
epoch: 6loss: tensor(1.0265)
epoch: 7loss: tensor(1.0237)
epoch: 8loss: tensor(1.0219)
epoch: 9loss: tensor(1.0207)
epoch: 10loss: tensor(1.0196)
epoch: 11loss: tensor(1.0191)
epoch: 12loss: tensor(1.0184)
epoch: 13loss: tensor(1.0178)
epoch: 14loss: tensor(1.0172)
epoch: 15loss: tensor(1.0173)
epoch: 16loss: tensor(1.0167)
epoch: 17loss: tensor(1.0166)
epoch: 18loss: tensor(1.0164)
epoch: 19loss: tensor(1.0163)
epoch: 20loss: tensor(1.0161)
epoch: 21loss: tensor(1.0160)
epoch: 22loss: tensor(1.0161)
epoch: 23loss: tensor(1.0157)
epoch: 24loss: tensor(1.0157)
epoch: 25loss: tensor(1.0157)
epoch: 26loss: tensor(1.0154)
epoch: 27loss: tensor(1.0153)
epoch: 28loss: tensor(1.0147)
epoch: 29loss: tensor(1.0129)
epoch: 30loss: tensor(1.0116)
epoch: 31loss: tensor(1.0093)
epoch: 32loss: tensor(1.0094)
epoch: 33loss: tensor(1.0051)
epoch: 34loss: tens